In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

results = pd.DataFrame(index=['PacketsSent', 'PacketsSentFLC', 'PacketsReceived', 'PacketsReceivedFLC', 
                              'PacketsLost(%)', 'PacketsLostFLC(%)', 'Jitter', 'JitterFLC', 'Latency', 'LatencyFLC',
                              'PowerConsumed(W)', 'PowerConsumedFLC(W)'])

wsn1 = pd.read_csv('wsn1.csv', usecols=['run', 'type', 'module', 'name', 'value', 'mean'])

run = "wsnSc1T"

for i in range(1, 11):
    packetSentCount = wsn1[(wsn1.run.str.startswith('wsnSc1T'+str(i)+'-1')) & (wsn1.type=='scalar') & 
                             (wsn1.module.str.startswith("Sc1.sensor")) & (wsn1.module.str.endswith("udp")) & 
                             (wsn1.name=='packetSent:count')]   

    packetReceivedCount = wsn1[(wsn1.run.str.startswith('wsnSc1T'+str(i)+'-1'))& (wsn1.type=='scalar') & 
                                 (wsn1.module.str.startswith("Sc1.sink")) & 
                                 (wsn1.module.str.endswith("udp")) & (wsn1.name=='packetReceived:count')]

    totalPkSentCount = sum(packetSentCount.value)
    totalPkReceivedCount = sum(packetReceivedCount.value)
    packetLoss = (totalPkSentCount - totalPkReceivedCount) * 100 / totalPkSentCount

    residualEnergy = wsn1[(wsn1.run.str.startswith('wsnSc1T'+str(i)+'-1')) & (wsn1.type=='scalar') & 
                            (wsn1.name=='residualEnergyCapacity:last') & 
                            (wsn1.module.str.endswith('energyStorage'))] 

    spentEnergyJ = 3000 - sum(residualEnergy.value)
    consumedPower = spentEnergyJ / 240

    latency = wsn1[(wsn1.run.str.startswith('wsnSc1T'+str(i)+'-1')) & (wsn1.type=='histogram') & 
                     (wsn1.name=='endToEndDelay:histogram')] 

    latencyMean = latency["mean"].mean()
    
    jitter = wsn1[(wsn1.run.str.startswith('wsnSc1T'+str(i)+'-1')) & (wsn1.type=='histogram') & 
                    (wsn1.name=='jitter:histogram')] 
    
    jitterMean = jitter["mean"].mean()

                        ########################################################
                        ########Simulation with FUZZY LOGIC CONTROLLER##########
                        ########################################################
    
    packetSentCountFLC = wsn1[(wsn1.run.str.startswith('wsnSc1T'+str(i)+'-0')) & (wsn1.type=='scalar') & 
                                (wsn1.module.str.startswith("Sc1.sensor")) & 
                                (wsn1.module.str.endswith("udp")) & (wsn1.name=='packetSent:count')] 

    packetReceivedCountFLC = wsn1[(wsn1.run.str.startswith('wsnSc1T'+str(i)+'-0'))& (wsn1.type=='scalar') & 
                                    (wsn1.module.str.startswith("Sc1.sink")) & 
                                    (wsn1.module.str.endswith("udp")) & (wsn1.name=='packetReceived:count')]

    totalPkSentCountFLC = sum(packetSentCountFLC.value)
    totalPkReceivedCountFLC = sum(packetReceivedCountFLC.value)
    packetLossFLC = (totalPkSentCountFLC - totalPkReceivedCountFLC) * 100 / totalPkSentCountFLC

    residualEnergyFLC = wsn1[(wsn1.run.str.startswith('wsnSc1T'+str(i)+'-0')) & (wsn1.type=='scalar') & 
                               (wsn1.name=='residualEnergyCapacity:last') & 
                               (wsn1.module.str.endswith('energyStorage'))] 

    spentEnergyJFLC = 3000 - sum(residualEnergyFLC.value)
    consumedPowerFLC = spentEnergyJFLC / 240

    latencyFLC = wsn1[(wsn1.run.str.startswith('wsnSc1T'+str(i)+'-0')) & (wsn1.type=='histogram') & 
                        (wsn1.name=='endToEndDelay:histogram')] 

    latencyMeanFLC = latencyFLC["mean"].mean()

    jitterFLC = wsn1[(wsn1.run.str.startswith('wsnSc1T'+str(i)+'-0')) & 
                       (wsn1.type=='histogram') & (wsn1.name=='jitter:histogram')] 

    jitterMeanFLC = jitterFLC["mean"].mean()   
    
                        ########################################################
                        ########                 RESULTS             ##########
                        ########################################################
            
    runDataCollector = [totalPkSentCount, totalPkSentCountFLC, totalPkReceivedCount, totalPkReceivedCountFLC,
                        packetLoss, packetLossFLC, jitterMean, jitterMeanFLC, latencyMean, latencyMeanFLC, 
                        consumedPower, consumedPowerFLC]
    results.insert(i-1, "T"+str(i), runDataCollector)
        
print(results)


                             T1          T2           T3           T4  \
PacketsSent          786.000000  901.000000  2461.000000  3067.000000   
PacketsSentFLC       803.000000  810.000000  2483.000000  3376.000000   
PacketsReceived      777.000000  894.000000  2454.000000  3055.000000   
PacketsReceivedFLC   796.000000  803.000000  2435.000000  3289.000000   
PacketsLost(%)         1.145038    0.776915     0.284437     0.391262   
PacketsLostFLC(%)      0.871731    0.864198     1.933145     2.577014   
Jitter                 0.027239    0.011044     0.013998     0.015151   
JitterFLC              0.025348    0.014176     0.018110     0.019635   
Latency                0.070166    0.035029     0.080581     0.107319   
LatencyFLC             0.053365    0.041555     0.065722     0.106677   
PowerConsumed(W)       1.601346    1.567247     1.669400     1.711498   
PowerConsumedFLC(W)    1.064861    1.044023     1.112220     1.152080   

                              T5            T6    